In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import pandas as pd
import json
import html5lib
import time
import glob

In [7]:
#setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 91.0.4472
[WDM] - Get LATEST driver version for 91.0.4472
[WDM] - Driver [C:\Users\delac\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


# Retrieve the CSV's and Convert to DataFrames

In [8]:
# Setup the URL to visit
url = 'https://aflsbkg.airfrance.fr/'

# Visit the URL
browser.visit(url)

In [23]:
# Store origin and Destination for Airfrance and KLM
AF_flight = ["ORD", "CDG"]
KL_flight =  ["ORD", "AMS"]

# Store the date Variable (day, month, year)
day = list(map(int, input("What day(s) do you wish to screen").split(",")))
month = input("Input the month in 3 word format (Ex = Apr)").title()
year = "2021"

What day(s) do you wish to screen13
Input the month in 3 word format (Ex = Apr)Jun


In [193]:
# navigate to the correct webpage for CSV download
browser.links.find_by_partial_text('Booking').click()
browser.links.find_by_partial_text('Search').click()

time.sleep(2.00)

# fill the flight number and date input field
browser.fill("segment.segOrigin", f"{AF_flight[0]}")
browser.fill("segment.segDestination", f"{AF_flight[1]}")
browser.fill("bookingSearch.legDepartureDate", f"{day[0]}/{month}/{year}")

# filter the search for the flight needed
browser.links.find_by_partial_text('Filter').click()

time.sleep(3.00)

# browser.click_link_by_id('FILTER_EXPAND_COLLAPSE')
#Visit inital page and scrape the first page data
html = browser.html
soup = bs(html, 'html.parser')
table = soup.find_all('table', class_= 'listTable')
df = pd.read_html(str(table))[0]

# Create a for loop to visit the other pages of data and concat with original df
num = 2
for y in range(10):
    try:
        browser.click_link_by_partial_href(f'newbookListPagingProcess.do?page={num}')
        html1 = browser.html
        soup1 = bs(html1, 'html.parser')
        table1 = soup1.find_all('table', class_= 'listTable')
        df2 = pd.read_html(str(table1))[0]
        df = pd.concat([df, df2], ignore_index=True)
        num += 1
    except:
        break
AF = df[(df.Prefix == '006') | (df.Prefix == '057')].drop(['Unnamed: 0'], axis = 1)

browser.links.find_by_partial_text('Booking').click()
browser.links.find_by_partial_text('Search').click()

time.sleep(3.00)

browser.fill("segment.segOrigin", f"{KL_flight[0]}")
browser.fill("segment.segDestination", f"{KL_flight[1]}")
browser.fill("bookingSearch.legDepartureDate", f"{day[0]}/{month}/{year}")


# filter the search for the flight needed
browser.links.find_by_partial_text('Filter').click()

time.sleep(3.00)

# browser.click_link_by_id('FILTER_EXPAND_COLLAPSE')
#Visit inital page and scrape the first page data
html = browser.html
soup = bs(html, 'html.parser')
table = soup.find_all('table', class_= 'listTable')
df = pd.read_html(str(table))[0]

# Create a for loop to visit the other pages of data and concat with original df
num = 2
for y in range(10):
    try:
        browser.click_link_by_partial_href(f'newbookListPagingProcess.do?page={num}')
        html1 = browser.html
        soup1 = bs(html1, 'html.parser')
        table1 = soup1.find_all('table', class_= 'listTable')
        df2 = pd.read_html(str(table1))[0]
        df = pd.concat([df, df2], ignore_index=True)
        num += 1
    except:
        break
KL = df[(df.Prefix == '057') |(df.Prefix == '074') ].drop(['Unnamed: 0'], axis = 1)

df_final = pd.concat([AF, KL], ignore_index = True)

C:\Users\delac\anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:485: FutureWarning: browser.find_link_by_partial_href is deprecated. Use browser.links.find_by_partial_href instead.
  FutureWarning,


# Filter the Dataframe

In [194]:
# Select the columns you want to display and store in a list

columns = ["Prefix", "AWB Number", "Origin","Destination","Commodity", \
           "Product","Pieces", "Weight","Volume","Issuer","Flight Info"]

# Filter the AWB's by  weight and volume 
df_final = df_final[(df_final.Weight.astype('float') > 1000) & (df_final.Volume.astype("float") > 10.00)][columns]

# Filter the AWB's by product
df_final = df_final[~df_final.Product.str.contains("S50|S51|S52|S53")].sort_values('Flight Info').reset_index(drop = True)

In [207]:
df_test = pd.DataFrame(columns = columns)
count = 0
for index, row in df_final.iterrows():
    if past_flight != row["Flight Info"]:
        if count != 0:
            df_test = df_test.append(pd.Series(name= " ")).fillna("")
        df_test = df_test.append(pd.Series(name= row["Flight Info"])).fillna("")
    else:
        df_test = df_test.append(row)
    past_flight = row["Flight Info"]
    count += 1
df_test

,Prefix,AWB Number,Origin,Destination,Commodity,Product,Pieces,Weight,Volume,Issuer,Flight Info
AF 0153,,,,,,,,,,,
1,057,92803255,ORD,LIS,GENERAL CARGO,R21,6.0,1350.0,11.88,EXFREIGHT ZETA INC*E,AF 0153
2,057,00189770,ORD,AMS,CONSOLIDATION,R21,,2424.0,11.68,CH ROBINSON ORD * EX,AF 0153
3,057,00210442,ORD,CDG,DANGEROUS GOODS,R91,40.0,11068.0,25.00,BDP INTERNATIONAL *,AF 0153
4,057,00640614,ORD,CDG,GENERAL CARGO,R21,26.0,1776.0,12.00,DACHSER TRANSPORT OF,AF 0153
,,,,,,,,,,,
AF 6737,,,,,,,,,,,
6,057,95738786,ORD,CDG,CONSOLIDATION C,R21,2.0,2775.0,20.00,BOLLORE LOGISTICS US,AF 6737
7,057,00551423,YUL,AMS,CONSOLIDATION,R21,7.0,3150.0,33.79,CH ROBINSON WORLWIDE,AF 6737
8,057,99843310,ORD,AMS,CONSOLIDATION,R21,2.0,1750.0,20.00,GEODIS USA INC,AF 6737


In [210]:
df_test.to_csv("test.csv")